In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: canaanbreiss1 (lm-informants). Use `wandb login --relogin` to force relogin


True

In [2]:
import pandas as pd 
import wandb
import numpy as np
from tqdm import tqdm


api = wandb.Api(timeout=50)
entity, project = "lm-informants", "maybefinal_four"  # set to your entity and project 
runs = api.runs(entity + "/" + project) 

In [4]:
df = pd.DataFrame()

In [5]:

summary_list, config_list, name_list = [], [], []
mega_df = pd.DataFrame()

all_data = []

for run in tqdm(runs): 
    
    # add all the keys that are logged that you want to download
    keys = [
        "step",
        "auc",
        ]
                
    df = pd.DataFrame()
    
    history = run.scan_history()
    
    history_df = pd.DataFrame(history)
                
    # filter ones that were killed
    if run.state != "finished":
        print("filtering run: ", run.path)
        continue
        
    # get the experiment config
    config = {k: v for k,v in run.config.items()
         if not k.startswith('_')}
    
    exp = {}
    
    # if key doesn't exist, set to nan (though this shouldn't happen after filtering empty runs)
    exp.update({f"{col}": history_df[~history_df[col].isnull()][col].values if col in history_df.columns else np.nan for col in keys})

    num_steps = len(exp[keys[0]])
    for k in keys:
        assert len(exp[k]) == num_steps
        
    # convert dict of lists to list of dicts
    results = [dict(zip(exp,t)) for t in zip(*exp.values())]
    
    for r in results:
        r.update({f"config/{key}": val for key, val in config.items()})
        r.update({f"strategy": run.name})
        r.update({f"wandb_id": run.path})
        
    all_data.extend(results)
    
    
# get status
    
mega_df = pd.DataFrame(all_data)


  0%|                                                                                                                                             | 1/2392 [00:00<18:15,  2.18it/s]

filtering run:  ['lm-informants', 'maybefinal_four', 'pin43enb']


  4%|██████                                                                                                                                     | 105/2392 [01:24<30:57,  1.23it/s]

filtering run:  ['lm-informants', 'maybefinal_four', 'gsz0vyiv']


 12%|████████████████                                                                                                                           | 277/2392 [03:35<30:02,  1.17it/s]

...

 26%|███████████████████████████████████▌                                                                                                       | 613/2392 [08:23<20:32,  1.44it/s]

filtering run:  ['lm-informants', 'maybefinal_four', '21gmklhf']


 48%|█████████████████████████████████████████████████████████████████▋                                                                        | 1139/2392 [15:44<16:05,  1.30it/s]

filtering run:  ['lm-informants', 'maybefinal_four', 'oiuh34ad']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2392/2392 [33:52<00:00,  1.18it/s]


In [5]:
display(mega_df)

,step,auc,config/run,config/n_init,config/strategy,config/tolerance,config/prior_prob,config/warm_start,config/feature_type,config/lexicon_file,config/converge_type,config/log_log_alpha_ratio,strategy,wandb_id
0,0.0,0.752471,1,0,kl_train_mixed,0.001,0.35,False,atr_harmony,None,symmetric,1.0,kl_train_mixed,"[lm-informants, maybefinal_four, b950h18d]"
1,1.0,0.752913,1,0,kl_train_mixed,0.001,0.35,False,atr_harmony,None,symmetric,1.0,kl_train_mixed,"[lm-informants, maybefinal_four, b950h18d]"
2,2.0,0.754696,1,0,kl_train_mixed,0.001,0.35,False,atr_harmony,None,symmetric,1.0,kl_train_mixed,"[lm-informants, maybefinal_four, b950h18d]"
3,3.0,0.757125,1,0,kl_train_mixed,0.001,0.35,False,atr_harmony,None,symmetric,1.0,kl_train_mixed,"[lm-informants, maybefinal_four, b950h18d]"
4,4.0,0.759138,1,0,kl_train_mixed,0.001,0.35,False,atr_harmony,None,symmetric,1.0,kl_train_mixed,"[lm-informants, maybefinal_four, b950h18d]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188545,145.0,1.000000,0,0,unif,0.001,0.10,False,atr_harmony,None,symmetric,0.5,unif,"[lm-informants, maybefinal_four, flciihpd]"
188546,146.0,1.000000,0,0,unif,0.001,0.10,False,atr_harmony,None,symmetric,0.5,unif,"[lm-informants, maybefinal_four, flciihpd]"
188547,147.0,1.000000,0,0,unif,0.001,0.10,False,atr_harmony,None,symmetric,0.5,unif,"[lm-informants, maybefinal_four, flciihpd]"
188548,148.0,1.000000,0,0,unif,0.001,0.10,False,atr_harmony,None,symmetric,0.5,unif,"[lm-informants, maybefinal_four, flciihpd]"


In [5]:
out_path = './runs.csv'
mega_df.to_csv(out_path)